In [1]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path
sys.path.append(str(Path("../../src").resolve()))

In [2]:
from utils.openai_lm import OpenAIModel
from utils.parser import load_prompt, load_jsonl, save_jsonl

In [3]:
# import random

# prompt_path = "prompt.txt"
# # passage_path = "../../outputs/passages.jsonl"
# passage_path = "../../scripts/download_papers/outputs/paragraphs.jsonl"

# prompt_template = load_prompt(prompt_path)
# doc_list = load_jsonl(passage_path)

# min_len = 100
# max_len = 400

# task_list = []

# para_list = sum([doc["paragraphs"] for doc in doc_list], [])
# para_list = [p for p in para_list if min_len < len(p["content"].split()) < max_len]
# print(f"Number of paragraphs: {len(para_list)}")

In [7]:
prompt_path = "prompt.txt"
para_path = "../../scripts/download_papers/outputs/paragraphs.jsonl"

prompt_template = load_prompt(prompt_path)
para_list = load_jsonl(para_path)

print(f"Number of paragraphs: {len(para_list)}")

Number of paragraphs: 78333


In [4]:
from utils.openai_lm import OpenAIModel
from utils.together_lm import TogetherModel

model_name = "gpt-4-0314"
# model_name = "gpt-35-turbo-0301"
# model_name = "togethercomputer/llama-2-7b-chat"
# model_name = "togethercomputer/llama-2-70b-chat"

if "gpt" in model_name:
    model = OpenAIModel(model_name=model_name, cache_file=None, model_mode="chat_gpt", api_type="azure")
else:
    model = TogetherModel(model_name=model_name, cache_file=None)

# output, response = model.generate("Hello, how are you?")
output, response = model.generate([{"role": "user", "content": "Hello, how are you?"}])
print(output)

As an AI, I do not have feelings or emotions, but I am here to help you. How can I assist you today?


In [10]:
import random

num_para = len(para_list)
# num_para = 100
random.seed(42)
para_sample = random.sample(para_list, num_para)
print(f"Number of paragraphs: {len(para_sample)}")

Number of paragraphs: 78333


In [11]:

input_messages_list = []
for para in para_sample:
    # print(para)
    title_section = para["title"] if para["section"] == "" else f"{para['title']}; {para['section']}"
    content = para["content"]
    prompt = prompt_template.substitute(title=title_section, content=content)
    para["input_messages"] = [{"role": "user", "content": prompt}]
    # input_messages_list.append([{"role": "user", "content": prompt}])
    # task_list.append(task)
    # print(prompt)

In [8]:
from tqdm import tqdm

output_path = f"outputs/instructions.{model_name.split('/')[-1]}.{num_para}.txt"
output_list = []
for i, para in enumerate(tqdm(para_sample)):
    try:
        input_messages = para["input_messages"]
        output, response = model._generate(input_messages)
        output_list.append({"instruction": output, "context": {"title": para["title"], "section": para["section"], "content": para["content"]}})
    except Exception as e:
        print(e)
    if len(output_list) % 100 == 1:
        save_jsonl(output_list, output_path)
        print(f"Number of outputs: {len(output_list)}")
save_jsonl(output_list, output_path)
print(f"Number of outputs: {len(output_list)}")

  0%|          | 0/100 [00:00<?, ?it/s]

  1%|          | 1/100 [00:05<09:29,  5.76s/it]

Number of outputs: 1


100%|██████████| 100/100 [07:11<00:00,  4.31s/it]

Number of outputs: 100
